## Задание 1 (10 баллов)

1) Обучите языковую модель с помощью GRPO на вот этом датасете - `notbadai/python_functions_reasoning`
Он чем-то похож на датасет из семинара, но вместо математических задач нужно написать код на питоне.

2) Измените system prompt под эту задачу.

3) Напишите аналогичные reward функции - одна на проверку формата (markdown wrappers ```python... ``` вокруг кода в конце), и на проверку что код работает (можете использовать import ast; ast.parse(string) для проверки)

Если модель сначала не может отвечать в нужно формате, то сделайте цикл SFT обучения на небольшом куске датасета.

GRPO в колабе может работать долго, поэтому не старайтесь пропустить весь датасет. Но обучайте хотя бы на 100 промптах (по несколько генераций на каждый промпт).

4) Оцените разницу в качестве на небольшом сабсете, прогнав изначальную модель, модель после SFT (если она есть) и GRPO модель. В качестве метрики используйте reward функции.

Можете взять любую языковую модель любого размера. Можете дообучать всю модель или же только адаптера поверх квантизированной модели.


## Imports

In [ ]:
!pip install trl
!pip install bitsandbytes
!pip install datasets
# удаляю вывод, т.к. много

In [2]:
import ast
import numpy as np
# import pandas as pd
import re
import torch
from datasets import load_dataset
from peft import LoraConfig, PeftModel
# from tqdm.autonotebook import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import GRPOConfig, GRPOTrainer, SFTConfig, SFTTrainer

## Definitions
Модель, промпт, функции

In [135]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
# https://huggingface.co/Qwen/Qwen2.5-0.5B-Instruct

device = 'cuda'

In [136]:
system_prompt = """
Your task is to write Python 3.9 code based on the user task.
Before outputting the result, give step-by-step reasoning.
Your code must be valid and wrapped into ```python *your code*```
"""

def chatml_format(example):
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": example['prompt']}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    return {
        "prompt": prompt
    }

In [137]:
pattern = r'```python\n(.*?)\n```'

def format_accuracy_func(completions, **kwargs):
    rewards = []
    pattern = r'```python\n(.*?)\n```'
    for response in completions:
        if re.search(pattern, response, re.S):
                                  # anyting in between tags
            rewards.append(1.0)
        else:
            rewards.append(0.0)
    return rewards

def answer_accuracy_func(completions, **kwargs):
    rewards = []
    pattern = r'```python\n(.*?)\n```'
    for response in completions:
        try:
            match_script = re.search(pattern, response, re.S)
            ast.parse(match_script.group(1)) # exception if not valid
            rewards.append(1.0)
        except Exception:
            rewards.append(0.0)
    return rewards

## Dataset

In [138]:
dataset = load_dataset("notbadai/python_functions_reasoning", "default")['train']
dataset = dataset.shuffle(seed=4)

data_train = dataset.select(range(0, 100))
data_test = dataset.select(range(100, 120))

In [139]:
data_test

Dataset({
    features: ['prompt', 'reasoning', 'answer', 'source'],
    num_rows: 20
})

In [140]:
data_train = data_train.map(chatml_format)
data_test = data_test.map(chatml_format)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [142]:
data_test

Dataset({
    features: ['prompt', 'reasoning', 'answer', 'source'],
    num_rows: 20
})

In [143]:
data_test[0]

{'prompt': '<|im_start|>system\n\nYour task is to write Python 3.9 code based on the user task. \nBefore outputting the result, give step-by-step reasoning. \nYour code must be valid and wrapped into ```python *your code*```\n<|im_end|>\n<|im_start|>user\n```python\ndef get_filtered_key(input_key, indices):\n    """Write a function to filter a tuple based on a list of indices. The function should return a new tuple containing only the elements from the input tuple that correspond to the specified indices.\n    """\n```<|im_end|>\n<|im_start|>assistant\n',
 'reasoning': 'To implement the function `get_filtered_key`, we need to follow these steps:\n\n1. **Understand the Inputs and Outputs:**\n   - The function takes two inputs:\n     - `input_key`: A tuple containing elements.\n     - `indices`: A list of integers representing the indices of the elements in `input_key` that we want to extract.\n   - The function should return a new tuple containing only the elements from `input_key` that

## Default model (only evaluation)

In [144]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16, # ещё поменьше сделать
    load_in_4bit=True # чтобы это работало
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

model = model.to(device)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [145]:
input_ids = tokenizer.batch_encode_plus(data_test['prompt'], return_tensors='pt', padding=True)

output = model.generate(
    input_ids['input_ids'].to(device), attention_mask=input_ids['attention_mask'].to(device),
    max_new_tokens=400, do_sample=True, temperature=1.5, pad_token_id=tokenizer.eos_token_id #**gen_kwargs
)

In [149]:
print(len(output))
output

20


tensor([[151645, 151645, 151645,  ...,   5500,    788,   1591],
        [151645, 151645, 151645,  ...,   1790,   1598,  42747],
        [151645, 151645, 151645,  ...,   3056,   1958,   1160],
        ...,
        [151645, 151645, 151645,  ..., 151645, 151645, 151645],
        [151645, 151645, 151645,  ...,   6678,      7,     15],
        [151645, 151645, 151645,  ..., 151645, 151645, 151645]],
       device='cuda:0')

In [150]:
completions = tokenizer.batch_decode(output, skip_special_tokens=False)

In [153]:
completions[0]
# есть какие-то странные штуки, но вроде при этом сам код тоже, см. ниже

'<|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_start|>system\n\nYour task is to write Python 3.9 code based on the user task. \nBefore outputting the result, give step-by-step reasoning. \nYour code must be valid and wrap

In [154]:
print(re.search(pattern, [re.sub('<\|im_end\|>+', '', i) for i in completions][0], re.S).group(1))

def get_filtered_key(input_key, indices):
    """Write a function to filter a tuple based on a list of indices. The function should return a new tuple containing only the elements from the input tuple that correspond to the specified indices.
    """


In [155]:
np.mean(format_accuracy_func(completions))

np.float64(0.75)

In [156]:
np.mean(answer_accuracy_func(completions))

np.float64(0.65)

Понятно, что с точки зрения реальной рабочести кода могут быть проблемы, но наши нехитрые тесты проходятся весьма неплохо из коробки

## GRPO

### Training

In [160]:
model_grpo = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    load_in_4bit=True
)

tokenizer_grpo = AutoTokenizer.from_pretrained(model_name)
tokenizer_grpo.pad_token = tokenizer.eos_token
tokenizer_grpo.padding_side = "left"

model_grpo = model_grpo.to(device)

model_grpo

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=896, out_features=896, bias=True)
          (k_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (v_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (o_proj): Linear4bit(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear4bit(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (

In [161]:
# добавляем адаптер ко всем линейным слоям
peft_config = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules="all-linear"
)

In [167]:
training_args = GRPOConfig(output_dir="Qwen/Qwen2.5-0.5B-Instruct-grpo",
                           logging_steps=1,
                           report_to="none",
                           num_generations=4,
                           num_train_epochs=1,
                           temperature=1.5,
                           label_names=["labels"])

In [168]:
trainer = GRPOTrainer(
    model=model_grpo,
    processing_class=tokenizer_grpo,
    reward_funcs=[answer_accuracy_func, format_accuracy_func],
    args=training_args,
    train_dataset=data_train,
    peft_config=peft_config
)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [169]:
trainer.train()

# не совсем понимаю, что означает 50: вроде в трейн датасете 100 примеров
# но когда я запускал с 8 генераций на каждый промпт, было 100 вместо 50

Step,Training Loss
1,0.016500
2,0.126700
3,0.063300
4,0.241400
5,0.018800
6,0.022000
7,0.029800
8,-0.000000
9,0.165300
10,0.082500


OutOfMemoryError: CUDA out of memory. Tried to allocate 594.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 256.12 MiB is free. Process 4082 has 14.49 GiB memory in use. Of the allocated memory 13.49 GiB is allocated by PyTorch, and 882.59 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# trainer.save_model('Qwen/Qwen2.5-0.5B-Instruct-grpo')

### Evaluation

In [171]:
input_ids_grpo = tokenizer_grpo.batch_encode_plus(data_test['prompt'], return_tensors='pt', padding=True)

output = model_grpo.generate(
    input_ids_grpo['input_ids'].to(device), \
    attention_mask=input_ids_grpo['attention_mask'].to(device),
    max_new_tokens=400, do_sample=True, temperature=1.5, \
    pad_token_id=tokenizer_grpo.eos_token_id #**gen_kwargs
)

In [172]:
completions_gpro = tokenizer_grpo.batch_decode(output, skip_special_tokens=False)

In [175]:
completions_gpro[0]
# опять: есть какие-то странные штуки, но вроде при этом сам код тоже, см. ниже

'<|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_start|>system\n\nYour task is to write Python 3.9 code based on the user task. \nBefore outputting the result, give step-by-step reasoning. \nYour code must be valid and wrap

In [176]:
print(re.search(pattern, [re.sub('<\|im_end\|>+', '', i) for i in completions_gpro][0], re.S).group(1))

def get_filtered_key(input_key, indices):
    """Write a function to filter a tuple based on a list of indices. The function should return a new tuple containing only the elements from the input tuple that correspond to the specified indices.
    """


In [177]:
np.mean(format_accuracy_func(completions_gpro))

np.float64(0.7)

In [178]:
np.mean(answer_accuracy_func(completions_gpro))

np.float64(0.6)

По сути это же самое, что было из коробки: не думаю, что статистически значимое отличие (хотя номинально качество стало ниже). Но с учётом того, какой маленький лосс с самого начала, получается, что обучение ничего не привносит в веса. Наверное, при другой процедуре обучения что-то бы изменилось

## Обучение SFT (неудачная попытка)
В конуе этого раздела вылезает непонятная ошибка. Но в общем модель и без SFT выдаёт в нормальном формате

In [157]:
model_sft = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    load_in_4bit=True
)

tokenizer_sft = AutoTokenizer.from_pretrained(model_name)
tokenizer_sft.pad_token = tokenizer.eos_token
tokenizer_sft.padding_side = "left"

model_sft = model_sft.to(device)

model_sft

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=896, out_features=896, bias=True)
          (k_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (v_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (o_proj): Linear4bit(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear4bit(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (

In [158]:
training_args_sft = SFTConfig(
    max_length=200,
    label_names=["labels"],
    report_to="none",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    logging_steps=1,
)

In [159]:
trainer = SFTTrainer(
    model=model_sft,
    processing_class=tokenizer_sft,
    train_dataset=data_train,
    args=training_args_sft,
)

Converting train dataset to ChatML:   0%|          | 0/100 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

KeyError: 'completion'